<a href="https://colab.research.google.com/github/MaliniVibin1502/SQL-projects/blob/main/Analysis_Chicago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is an analysis of Chicago area schools, socio-economic factors and the crime data. I have used SQL magic and pandas for the analysis of data within the dataset by loading the csv files to a database called Chicago.

In [34]:
!pip install sqlite3
import sqlite3
import pandas as pd

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [35]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [36]:
con = sqlite3.connect("Chicago.db")
cur = con.cursor()

In [37]:
%sql sqlite:///Chicago.db

In [38]:
df = pd.read_csv("https://raw.githubusercontent.com/MaliniVibin1502/SQL-projects/main/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pd.read_csv("https://raw.githubusercontent.com/MaliniVibin1502/SQL-projects/main/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("https://raw.githubusercontent.com/MaliniVibin1502/SQL-projects/main/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

566

Find the total number of crimes recorded in the CRIME table.

In [39]:
%%sql
SELECT COUNT(DISTINCT(CASE_NUMBER)) AS TOTAL_CASES FROM CHICAGO_CRIME_DATA

 * sqlite:///Chicago.db
Done.


TOTAL_CASES
533


List community area names and numbers with per capita income less than 11000.

In [40]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


List all case numbers for crimes involving minors?

In [41]:
%%sql
SELECT COUNT(CASE_NUMBER) AS TOTAL_MINOR_CASES FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR'

 * sqlite:///Chicago.db
Done.


TOTAL_MINOR_CASES
2


List all kidnapping crimes involving a child?

In [42]:
%%sql
SELECT COUNT(CASE_NUMBER) AS TOTAL_CHILD_ADBUCTION FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = "KIDNAPPING" AND DESCRIPTION LIKE 'CHILD%'

 * sqlite:///Chicago.db
Done.


TOTAL_CHILD_ADBUCTION
1


List the kind of crimes that were recorded at schools.

In [43]:
%%sql
SELECT COUNT(CASE_NUMBER) AS TOTAL_CASES_REPORTED_AT_SCHOOL FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE "SCHOOL%"

 * sqlite:///Chicago.db
Done.


TOTAL_CASES_REPORTED_AT_SCHOOL
12


List the type of schools along with the average safety score for each type.

In [44]:
%%sql
SELECT "Elementary, Middle, or High School", ROUND(AVG(SAFETY_SCORE),2) AS AVERAGE_SCORE FROM CHICAGO_PUBLIC_SCHOOLS_DATA GROUP BY "Elementary, Middle, or High School" ORDER BY SAFETY_SCORE

 * sqlite:///Chicago.db
Done.


"Elementary, Middle, or High School",AVERAGE_SCORE
HS,49.62
MS,48.0
ES,49.52


List 5 community areas with highest % of households below poverty line

In [45]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


Which community area is most crime prone?

In [46]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COUNT(CASE_NUMBER) AS TOTAL_CASES FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY TOTAL_CASES DESC LIMIT 1

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NUMBER,TOTAL_CASES
25.0,43


Use a sub-query to find the name of the community area with highest hardship index

In [47]:
%%sql
SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM CENSUS_DATA WHERE HARDSHIP_INDEX=(SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


 Use a sub-query to determine the Community Area Name with most number of crimes?

In [52]:
%%sql
SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(CASE_NUMBER) DESC LIMIT 1)

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NAME
Austin
